In [1]:
import pandas as pd
import numpy as np
%pylab inline

Populating the interactive namespace from numpy and matplotlib


## Entities

In [2]:
docs_ents = pd.read_csv("../download/documents_entities.csv")
docs_ents.head()

,document_id,entity_id,confidence_level
0,1524246,f9eec25663db4cd83183f5c805186f16,0.672865
1,1524246,55ebcfbdaff1d6f60b3907151f38527a,0.399114
2,1524246,839907a972930b17b125eb0247898412,0.392096
3,1524246,04d8f9a1ad48f126d5806a9236872604,0.213996
4,1617787,612a1d17685a498aff4f036c1ee02c16,0.386193


In [4]:
docs_ents['value']=docs_ents['confidence_level']
docs_ents['value2']=np.square(docs_ents['value'])
docs_ents.drop(['confidence_level'], axis=1, inplace=True)
docs_ents.head()

,document_id,entity_id,value,value2
0,1524246,f9eec25663db4cd83183f5c805186f16,0.672865,0.452748
1,1524246,55ebcfbdaff1d6f60b3907151f38527a,0.399114,0.159292
2,1524246,839907a972930b17b125eb0247898412,0.392096,0.153739
3,1524246,04d8f9a1ad48f126d5806a9236872604,0.213996,0.045794
4,1617787,612a1d17685a498aff4f036c1ee02c16,0.386193,0.149145


In [5]:
docs_norms2 = pd.DataFrame({'norm2':docs_ents.groupby(['document_id'])['value2'].sum()}).reset_index()
docs_ents.drop(['value2'], axis=1, inplace=True)
docs_norms2.head()

,document_id,norm2
0,1,0.231968
1,2,0.718268
2,3,0.103002
3,4,1.508028
4,5,0.327309


### Train

In [6]:
page_addoc_dist = pd.read_csv("../generated/final/events_clicks_CTR_addocs_train.csv", usecols=['document_id', 'addoc_id'])
page_addoc_dist.count()

document_id    87141731
addoc_id       87141731
dtype: int64

In [7]:
page_addoc_dist = pd.DataFrame({'size':page_addoc_dist.groupby(['document_id', 'addoc_id']).size()}).reset_index().drop(['size'], axis=1)
page_addoc_dist.count()

document_id    28491064
addoc_id       28491064
dtype: int64

In [8]:
len(page_addoc_dist.document_id.unique()), len(page_addoc_dist.addoc_id.unique())

(756510, 164476)

In [13]:
page_addoc_diffs = page_addoc_dist.merge(docs_ents, left_on='addoc_id', right_on='document_id')
page_addoc_diffs.drop(['document_id_y'], axis=1, inplace=True)
page_addoc_diffs.rename(columns={'document_id_x':'document_id'}, inplace=True)
page_addoc_diffs = page_addoc_diffs.merge(docs_ents, on=['document_id', 'entity_id'])
page_addoc_diffs['value_2xy'] = 2*page_addoc_diffs['value_x']*page_addoc_diffs['value_y']
page_addoc_diffs.drop(['entity_id', 'value_x', 'value_y'], axis=1, inplace=True)
page_addoc_diffs = pd.DataFrame({'value_2xy':page_addoc_diffs.groupby(['document_id', 'addoc_id'])['value_2xy'].sum()}).reset_index()

page_addoc_diffs.head()

,document_id,addoc_id,value_2xy
0,63,2348788,0.590672
1,122,1009076,0.110330
2,222,1319517,0.324632
3,222,1804537,0.390811
4,357,2022997,0.122662


In [14]:
page_addoc_diffs.count()

document_id    160408
addoc_id       160408
value_2xy      160408
dtype: int64

In [19]:
page_addoc_dist = page_addoc_dist.merge(docs_norms2, on=['document_id'])
page_addoc_dist = page_addoc_dist.merge(docs_norms2, left_on=['addoc_id'], right_on=['document_id'])
page_addoc_dist.rename(columns={'document_id_x':'document_id'}, inplace=True)
page_addoc_dist.drop(['document_id_y'], axis=1, inplace=True)
page_addoc_dist = page_addoc_dist.merge(page_addoc_diffs, on=['document_id', 'addoc_id'], how='left').fillna(0)
page_addoc_dist['dist_ents'] = np.sqrt( page_addoc_dist['norm2_x'] + page_addoc_dist['norm2_y'] - page_addoc_dist['value_2xy'] )
page_addoc_dist.drop(['norm2_x', 'norm2_y', 'value_2xy'], axis=1, inplace=True)

page_addoc_dist.head()

,document_id,addoc_id,dist_ents
0,2,1489042,1.310801
1,452,1489042,1.274215
2,515,1489042,1.169839
3,3730,1489042,1.184130
4,7794,1489042,1.043407


In [20]:
page_addoc_dist.to_csv("../generated/final/page_addoc_ents_dist.csv", index=False)

In [21]:
page_addoc_dist.count()

document_id    13628638
addoc_id       13628638
dist_ents      13628638
dtype: int64

### Test

In [ ]:
page_addoc_dist = pd.read_csv("../generated/final/events_clicks_CTR_addocs_test.csv", usecols=['document_id', 'addoc_id'])
page_addoc_dist.count()

In [ ]:
page_addoc_dist = pd.DataFrame({'size':page_addoc_dist.groupby(['document_id', 'addoc_id']).size()}).reset_index().drop(['size'], axis=1)
page_addoc_dist.count()

In [ ]:
len(page_addoc_dist.document_id.unique()), len(page_addoc_dist.addoc_id.unique())

In [ ]:
page_addoc_diffs = page_addoc_dist.merge(docs_cats, left_on='addoc_id', right_on='document_id')
page_addoc_diffs.rename(columns={'document_id_x':'document_id'}, inplace=True)
page_addoc_diffs.drop(['document_id_y'], axis=1, inplace=True)
page_addoc_diffs = page_addoc_diffs.merge(docs_cats, on=['document_id', 'category_id'])
page_addoc_diffs['value_2xy'] = 2*page_addoc_diffs['value_x']*page_addoc_diffs['value_y']
page_addoc_diffs.drop(['category_id', 'value_x', 'value_y'], axis=1, inplace=True)
page_addoc_diffs = pd.DataFrame({'value_2xy':page_addoc_diffs.groupby(['document_id', 'addoc_id'])['value_2xy'].sum()}).reset_index()

page_addoc_diffs.head()

In [ ]:
page_addoc_diffs.count()

In [ ]:
page_addoc_dist = page_addoc_dist.merge(docs_norms2, on=['document_id'])
page_addoc_dist = page_addoc_dist.merge(docs_norms2, left_on=['addoc_id'], right_on=['document_id'])
page_addoc_dist.rename(columns={'document_id_x':'document_id'}, inplace=True)
page_addoc_dist.drop(['document_id_y'], axis=1, inplace=True)
page_addoc_dist = page_addoc_dist.merge(page_addoc_diffs, on=['document_id', 'addoc_id'], how='left').fillna(0)
page_addoc_dist['dist_cats'] = np.sqrt( page_addoc_dist['norm2_x'] + page_addoc_dist['norm2_y'] - page_addoc_dist['value_2xy'] )
page_addoc_dist.drop(['norm2_x', 'norm2_y', 'value_2xy'], axis=1, inplace=True)

page_addoc_dist.head()

In [ ]:
page_addoc_dist.to_csv("../generated/final/page_addoc_cats_dist_test.csv", index=False)

In [ ]:
page_addoc_dist.count()